In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
import torch.nn.functional as F
import xgboost as xgb
from sklearn.model_selection import  StratifiedKFold
from sklearn.base import clone
import torch.utils.data as Data


In [90]:

# 定义神经网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.temporal = nn.Conv1d(in_channels=256,out_channels=256,kernel_size=3)
        self.temporal2 = nn.Conv1d(in_channels=256,out_channels=256,kernel_size=3)

        self.conv1 = nn.Conv2d(in_channels=1,out_channels=3,kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv2d(3, 3, 5)
        self.fc1 = nn.Linear(256*38, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 6)


    def forward(self, x):
        x = self.pool1(F.relu(self.temporal(x)))
        x = self.pool1(F.relu(self.temporal2(x)))
        x = torch.unsqueeze(x,1)
        #x = self.pool(F.relu(self.conv1(x)))
        #x = self.pool(F.relu(self.conv2(x)))


        x = x.view(-1,256*38)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)



        return x


class CBAMLayer(nn.Module):
    def __init__(self, channel, reduction=16, spatial_kernel=7):
        super(CBAMLayer, self).__init__()

        # channel attention 压缩H,W为1
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        # shared MLP
        self.mlp = nn.Sequential(
            # Conv2d比Linear方便操作
            # nn.Linear(channel, channel // reduction, bias=False)
            nn.Conv2d(channel, channel // reduction, 1, bias=False),
            # inplace=True直接替换，节省内存
            nn.ReLU(inplace=True),
            # nn.Linear(channel // reduction, channel,bias=False)
            nn.Conv2d(channel // reduction, channel, 1, bias=False)
        )

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        max_out = self.mlp(self.max_pool(x))
        avg_out = self.mlp(self.avg_pool(x))
        channel_out = self.sigmoid(max_out + avg_out)
        x = channel_out * x

        return x


In [19]:
import scipy.io as scio
clean_path = "D:\\BaiduSyncdisk\\code"
import scipy.io as scio
import os
import numpy as np
forward = int(40)
backward = int(120)
#onset_time = {"HS44":0.368,"HS45":0.151,"HS47":0.156,"HS48":0.172,"HS50":0.357,"HS54":0.322}
onset_time = {"HS44":0.368,"HS45":0.151,"HS47":0.156,"HS48":0.172,"HS50":0.357,"HS54":0.322}

HS = 54
file_name = "HS"+str(HS)+"_Block_overt_covert.mat"
z3 = scio.loadmat(os.path.join(clean_path,file_name))
z3 = z3["Alldata"][0][0]

onset = int(np.floor(100 * onset_time["HS" + str(HS)])) + 150
x1 = z3["ECoG_"+"overt_ba"][:,:,onset-forward:onset+backward]
x2 = z3["ECoG_"+"overt_da"][:,:,onset-forward:onset+backward]
x3 = z3["ECoG_"+"overt_ga"][:,:,onset-forward:onset+backward]
x4 = z3["ECoG_"+"overt_bu"][:,:,onset-forward:onset+backward]
x5 = z3["ECoG_"+"overt_du"][:,:,onset-forward:onset+backward]
x6 = z3["ECoG_"+"overt_gu"][:,:,onset-forward:onset+backward]

print(x1.shape)
print(x2.shape)


(53, 256, 160)
(57, 256, 160)


In [87]:
net=Net()
print(net(torch.Tensor(x2[0])).shape)



torch.Size([256, 38])
torch.Size([256, 1, 38])
torch.Size([1, 6])


In [91]:

# 加载数据集
net=Net()
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()


x = np.concatenate((x1,x2,x3,x4,x5,x6),axis=0)
y = np.ones([len(x1)+len(x2)])
y[:len(x1)]=0
print(y.shape)
y = np.concatenate((0*np.ones([len(x1)]),np.ones([len(x2)]),2*np.ones([len(x3)]),3*np.ones([len(x4)]),4*np.ones([len(x5)]),5*np.ones([len(x6)])),axis=0)
BATCH_SIZE = 4
trainset = Data.TensorDataset(torch.Tensor(x) ,torch.LongTensor(y))
loader = Data.DataLoader(
    dataset=trainset,
    batch_size=BATCH_SIZE,
    shuffle=True
    )
# 五折交叉验证
kf = KFold(n_splits=10,shuffle=True)
for train_index, test_index in kf.split(trainset):
    train_subset = torch.utils.data.Subset(trainset, train_index)
    test_subset = torch.utils.data.Subset(trainset, test_index)

    trainloader = torch.utils.data.DataLoader(train_subset, batch_size=4, shuffle=True, num_workers=1)
    testloader = torch.utils.data.DataLoader(test_subset, batch_size=4, shuffle=False, num_workers=1)

    net = Net()
    optimizer = optim.Adam(net.parameters(), lr=0.01)
    for epoch in range(2):  # loop over the dataset multiple times
        running_loss = 0.0
        correct = 0
        total = 0

        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            correct += (predicted == labels).sum().item()
            # print statistics
            running_loss += loss.item()
            total += labels.size(0)

            if i % 20 == 19:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
                print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

                running_loss = 0.0

        # 设置模型为评估模式
    net.eval()

    correct = 0
    total = 0
    correct_label = 0
    # 不计算梯度
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            #print(outputs.data)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            correct_label += torch.sum(labels)
    print('True on the test images: %d %%' % (100 * correct_label / total))
    print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))



(110,)
[1,    20] loss: 0.082
Accuracy of the network on the test images: 12 %
[1,    40] loss: 0.018
Accuracy of the network on the test images: 15 %
[1,    60] loss: 0.022
Accuracy of the network on the test images: 15 %
[2,    20] loss: 0.018
Accuracy of the network on the test images: 22 %
[2,    40] loss: 0.018
Accuracy of the network on the test images: 20 %
[2,    60] loss: 0.018
Accuracy of the network on the test images: 20 %
True on the test images: 255 %
Accuracy of the network on the test images: 8 %
[1,    20] loss: 0.062
Accuracy of the network on the test images: 15 %
[1,    40] loss: 0.021
Accuracy of the network on the test images: 20 %
[1,    60] loss: 0.022
Accuracy of the network on the test images: 18 %
[2,    20] loss: 0.020
Accuracy of the network on the test images: 16 %
[2,    40] loss: 0.018
Accuracy of the network on the test images: 14 %
[2,    60] loss: 0.018
Accuracy of the network on the test images: 13 %
True on the test images: 250 %
Accuracy of the net

In [ ]:
fish's z Transform
